In [18]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

%load_ext autoreload
%autoreload 2

from utils import helper, config, rayer, kaggle_dataset_helper

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from ml.models.base.tune.slug_lightgbm import SlugLGBM

from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common

import ray


import time
import pandas as pd
import pyarrow.fs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
rayer.get_global_cluster()

In [11]:
df = helper.get_tokamat_dataset()
df = df.reset_index()

df = common.label_encode(df)
df = df.fillna(-1)

/mnt/c/Users/rwmas/GitHub/xai/python-asd/xai/notebooks/tune_sklearn/../../utils


In [12]:
potential_targets = ['WTOT', 'WTH', 'PLTH']
y = df[potential_targets[0]]

X = df[df.columns[~df.columns.isin(potential_targets)]]
X = X.drop(['TOK_ID', 'LCUPDATE', 'DATE', 'NEL', 'ENBI'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [13]:
r2_scoring = make_scorer(score_func=r2_score, greater_is_better=False)

In [19]:
# regression, regression_l1, poisson, check for more options here: https://lightgbm.readthedocs.io/en/latest/Parameters.html
objective = "poisson" # 

slug_lgbm_1 = SlugLGBM(name='tune_slug_lgbm_1', objective=objective,
                                                pred_class='regression',
                                                #n_estimators=100,
                                                #max_depth=30,
                                                score_func=None,
                                                #metric_func=mean_squared_error, 
                                                n_trials=2)

In [20]:
slug_lgbm_1.fit(X_train, X_test, y_train, y_test)

(run pid=27879) /home/rayuser/.local/lib/python3.8/site-packages/ray/tune/execution/trial_runner.py:363: UserWarning: fail_fast='raise' detected. Be careful when using this mode as resources (such as Ray processes, file descriptors, and temporary files) may not be cleaned up properly. To use a safer mode, use fail_fast=True.
(run pid=27879)   warnings.warn(


(<ray.tune.trainable.util._Trainable object at 0x7f50a0d6e700> pid=12967, ip=192.168.0.38) [LightGBM] [Warning] lambda_l1 is set=0.017516091316022286, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.017516091316022286
(<ray.tune.trainable.util._Trainable object at 0x7f50a0d6e700> pid=12967, ip=192.168.0.38) [LightGBM] [Warning] bagging_fraction is set=6.849051624705767e-05, subsample=1.0 will be ignored. Current value: bagging_fraction=6.849051624705767e-05
(<ray.tune.trainable.util._Trainable object at 0x7f50a0d6e700> pid=12967, ip=192.168.0.38) [LightGBM] [Warning] lambda_l2 is set=0.009801405189087791, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.009801405189087791
(<ray.tune.trainable.util._Trainable object at 0x7f50a0d6e700> pid=12967, ip=192.168.0.38) [LightGBM] [Warning] learning_rate is set=0.1, eta=5.823615884845211e-06 will be ignored. Current value: learning_rate=0.1
(<ray.tune.trainable.util._Trainable object at 0x7f4460fea6a0> pid=28565) [LightGBM] 

(run pid=27879) 2022-12-06 10:27:05,876	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'cv': KFold(n_splits=3, random_state=None, shuffle=False), 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'scoring/score': <function _passthrough_scorer at 0x7fca4ea4cca0>}
(run pid=27879) 2022-12-06 10:27:10,468	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'cv': KFold(n_splits=3, random_state=None, shuffle=False), 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'scoring/score': <function _passthrough_scorer at 0x7fca4ea4cca0>}


[LightGBM] [Warning] lambda_l1 is set=0.016418899137805174, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.016418899137805174
[LightGBM] [Warning] bagging_fraction is set=1.690572438932412e-06, subsample=1.0 will be ignored. Current value: bagging_fraction=1.690572438932412e-06
[LightGBM] [Warning] lambda_l2 is set=0.0076628565313817355, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0076628565313817355
[LightGBM] [Warning] learning_rate is set=0.1, eta=0.639585350574135 will be ignored. Current value: learning_rate=0.1


In [21]:
slug_lgbm_1.scores

[0.978459164158111, 0.969067550719808]

In [23]:
slug_lgbm_1.score(X_test, y_test)

0.969067550719808